# FairSeqを用いた機械翻訳
https://github.com/pytorch/fairseq

## 1. ライブラリのインストール

In [ ]:
# Pythonのバージョンが3.6以上であることを確認
import sys
print(sys.version)

# PyTorchのバージョンが1.5.0以上であることを確認
import torch
print(torch.__version__)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
1.9.0+cu111


In [ ]:
# インストール
! pip install fairseq

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 145 kB 42.1 MB/s 
     |████████████████████████████████| 90 kB 8.9 MB/s 
     |████████████████████████████████| 74 kB 3.3 MB/s 
     |████████████████████████████████| 112 kB 48.1 MB/s 
     |████████████████████████████████| 596 kB 38.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=d283093137b5ca1c94dd989433f37a3fe3515065b01718eb03d5c1bc71552c20
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# fairseqのバージョン確認
! pip show fairseq

Name: fairseq
Version: 0.10.2
Summary: Facebook AI Research Sequence-to-Sequence Toolkit
Home-page: https://github.com/pytorch/fairseq
Author: None
Author-email: None
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: torch, cython, dataclasses, regex, numpy, tqdm, sacrebleu, cffi, hydra-core
Required-by: 


## 2. 日英対訳データの準備
https://github.com/odashi/small_parallel_enja

In [ ]:
# ダウンロード
! git clone https://github.com/odashi/small_parallel_enja.git

Cloning into 'small_parallel_enja'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.


In [ ]:
# データサイズ（行数）の確認
! wc -l ./small_parallel_enja/*.en ./small_parallel_enja/*.ja

    500 ./small_parallel_enja/dev.en
    500 ./small_parallel_enja/test.en
  50000 ./small_parallel_enja/train.en
    500 ./small_parallel_enja/dev.ja
    500 ./small_parallel_enja/test.ja
  50000 ./small_parallel_enja/train.ja
 102000 total


In [ ]:
# データ内容の確認
! head -3 ./small_parallel_enja/train.en ./small_parallel_enja/train.ja

==> ./small_parallel_enja/train.en <==
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .

==> ./small_parallel_enja/train.ja <==
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。


普通は次に単語分割をしますが、今回のデータは分割済みなのでスキップします。

単語分割にはSentencePieceなどのツールを使います。

https://github.com/google/sentencepiece

In [ ]:
# データ形式の変更
DATA = "/content/small_parallel_enja"
! fairseq-preprocess --source-lang en --target-lang ja --trainpref $DATA/train --validpref $DATA/dev --testpref $DATA/test

2021-10-22 02:24:44 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='en', srcdict=None, target_lang='ja', task='translation', tensorboard_logdir=None, testpref='/content/small_parallel_enja/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, tp

## 3. 翻訳器の訓練

In [ ]:
! fairseq-train data-bin --arch transformer \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --clip-norm 0.0 \
    --optimizer adam --max-tokens 4096 --max-epoch 5

2021-10-22 02:27:51 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_ba

## 4. 翻訳器の評価

In [ ]:
# 評価用データの翻訳
! fairseq-generate data-bin --path checkpoints/checkpoint_best.pt --batch-size 128 --beam 5 > result.txt
! head -20 result.txt

  0% 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
2021-10-22 02:40:51 | INFO | fairseq_cli.generate | Namespace(all_gather_list_size=16384, batch_size=128, batch_size_valid=128, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoding_format=None, device_id=0,

In [ ]:
# 出力ファイルから生成文を抽出
! grep "^H-" result.txt | sort -V | cut -f3 > result.ja.txt
! head result.ja.txt

彼 ら は その 話 を し た 。
彼 は 泳 ぐ 必要 は な かっ た 。
彼 は 妹 と 同じ くらい 背 が 高 く な い 。
１０ 時 前 に １０ 分 前 に 帰 っ て い る 。
もう 少し 眠 っ て しま っ た 。
彼女 は 私 たち の 隣 に 住 ん で い る 。
あなた の 答え に 答え る こと が でき る 。
私 は その 村 に 住 ん で い る 人 だ 。
私 たち は この 試合 に 勝 っ て い る 。
これ を どう し て くれ ま せ ん か 。


In [ ]:
# BLEUスコアの計算（10エポック学習すると30点ぐらいのそこそこ良い翻訳になります）
! fairseq-score --sys result.ja.txt --ref $DATA/test.ja

Namespace(ignore_case=False, order=4, ref='/content/small_parallel_enja/test.ja', sacrebleu=False, sentence_bleu=False, sys='result.ja.txt')
BLEU4 = 20.56, 53.0/28.7/15.7/8.9 (BP=0.957, ratio=0.958, syslen=5400, reflen=5635)


In [ ]:
! head result.ja.txt $DATA/test.ja $DATA/test.en

==> result.ja.txt <==
彼 ら は その 話 を し た 。
彼 は 泳 ぐ 必要 は な かっ た 。
彼 は 妹 と 同じ くらい 背 が 高 く な い 。
１０ 時 前 に １０ 分 前 に 帰 っ て い る 。
もう 少し 眠 っ て しま っ た 。
彼女 は 私 たち の 隣 に 住 ん で い る 。
あなた の 答え に 答え る こと が でき る 。
私 は その 村 に 住 ん で い る 人 だ 。
私 たち は この 試合 に 勝 っ て い る 。
これ を どう し て くれ ま せ ん か 。

==> /content/small_parallel_enja/test.ja <==
彼 ら は つい に それ が 真実 だ と 認め た 。
彼 は 水泳 が 得意 で は な かっ た 。
彼 は お 姉 さん に 劣 ら ず 親切 だ 。
１０ 時 前 に 戻 ら な けれ ば な ら な い 。
成功 を 祈 る わ 。
彼女 は 私 たち の 隣 の 家 に す ん で い る 。
あなた に 返事 を し よ う と し て い る ところ で す 。
私 は 刹那 的 な 生き 方 を し て い る 人間 で す 。
この 試合 は いただ き だ 。
こんな こと を し た 理由 を 言 い な さ い 。

==> /content/small_parallel_enja/test.en <==
they finally acknowledged it as true .
he didn 't care for swimming .
he is no less kind than his sister .
you must be back before ten .
break a leg .
she lives next door to us .
i 'm about to tell you the answer .
i 'm a person who lives for the moment .
we have this game on ice .
will you give me your reasons for doing this ?
